In [18]:
%reload_ext autoreload
%autoreload 2
%reload_ext line_profiler
%matplotlib inline
from tqdm import tnrange, tqdm_notebook

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import os
import seaborn as sns
import sys

import re

In [9]:
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)

In [11]:
!ls -alF ../input

total 125032
drwxrwxr-x 2 chrisyang chrisyang      4096 12월 28 14:28 ./
drwxrwxr-x 6 chrisyang chrisyang      4096 12월 28 14:37 ../
-rw-rw-r-- 1 chrisyang chrisyang   1167417 12월 16 13:15 public_test.csv
-rw-rw-rw- 1 chrisyang chrisyang 126845824 11월 28 22:04 train.csv


In [14]:
DATASET_PATH = '../input'

train_df = pd.read_csv(os.path.join(DATASET_PATH, 'train.csv'))
test_df = pd.read_csv(os.path.join(DATASET_PATH, 'public_test.csv'))

In [15]:
train_df.head()

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0
2,2,2017-01,안녕하십니까 고객님. XXX은행입니다.금일 납부하셔야 할 금액은 153600원 입니...,0
3,4,2017-01,XXX 고객님안녕하세요XXX은행 XXX지점입니다지난 한 해 동안 저희 XXX지점에 ...,0
4,5,2017-01,1월은 새로움이 가득XXX입니다.올 한해 더 많이행복한 한해되시길바랍니다,0


In [16]:
test_df.head()

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...
2,340002,2019-01,XXX 고객님 안녕하세요올해는 미세먼지가 유난인거 같습니다.엊그제 새해가 시작된거같...
3,340003,2019-01,XXX 고객님찾아온 행운을 잡으셨나요? 못잡으셨다면 이번에 다시 잡으시길 기원합니다...
4,340004,2019-01,XXX 고객님새해 복 많이 받으세요 XXX은행 코스트코 퇴직연금 담당자입니다. 고...


In [19]:
from soynlp.hangle import decompose

doublespace_pattern = re.compile('\s+')

def jamo_sentence(sent):

    def transform(char):
        if char == ' ':
            return char
        cjj = decompose(char)
        try:
            len(cjj)
        except:
            return ' '
        if len(cjj) == 1:
            return cjj
        cjj_ = ''.join(c if c != ' ' else '' for c in cjj)
        return cjj_

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_
# 'ㅇㅓ-ㅇㅣ-ㄱㅗ- ㅋㅔㄱㅋㅔㄱ ㅇㅏ-ㅇㅣ-ㄱㅗ-ㅇㅗ-'
def jamo_sentence(sent):
    sent = sent.replace('XXX', 'X')
    def transform(char):
        if char == ' ':
            return char
        elif char == 'X':
            return ' X'
        elif char == '.':
            return '. '
        else:
            return char

    sent_ = ''.join(transform(char) for char in sent)
    sent_ = doublespace_pattern.sub(' ', sent_)
    return sent_

In [20]:
jamo_sentence(train_df.loc[0, 'text'])

' X은행성산 X팀장입니다. 행복한주말되세요'

In [21]:
train_df['jamo'] = train_df['text'].apply(lambda x: jamo_sentence(x))
test_df['jamo'] = test_df['text'].apply(lambda x: jamo_sentence(x))

In [22]:
# raw_corpus_fname = '\n'.join(pd.concat([train_df['jamo'], test_df['jamo']]))
# file=open('/content/textfile.txt','w')
# file.write(raw_corpus_fname)
# file.close()

In [27]:
# import fasttext
# model = fasttext.train_unsupervised('/content/textfile.txt', 
#                                     model='cbow' ,
#                                     loss = 'hs',        # hinge loss
#                                     ws=1,               # window size
#                                     lr = 0.01,          # learning rate
#                                     dim = 200,          # embedding dimension
#                                     epoch = 5,          # num of epochs
#                                     min_count = 10,     # minimum count of subwords
#                                     # encoding = 'utf-8', # input file encoding
#                                     thread = -1          # num of threads
#                                 )

In [ ]:
# model.save_model("/content/model_file.bin")
# model.save_model("/content/drive/My Drive/model_file.bin")